In [8]:
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KernelDensity, KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

data = datasets.fetch_openml('mnist_784', version=1, return_X_y=True)
pixel_values, targets = data
targets = targets.astype(int)

# 划分训练集和测试集
X_train = pixel_values.to_numpy()[:60000, :]
y_train = targets[:60000]
X_test = pixel_values.to_numpy()[60000:, :]
y_test = targets[60000:]

# Parzen窗估计 
kde = KernelDensity(kernel='gaussian', bandwidth=3.0).fit(X_train)
log_densities = kde.score_samples(X_train[:10, :])  # 这里只是计算训练数据前10个样本的log密度，仅为示例
print("Log densities of first 10 training samples:", log_densities)

# KNN分类
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test[:10000, :]) 
accuracy = accuracy_score(y_test[:10000], y_pred)
print(f"KNN Accuracy : {accuracy:.4f}")

/opt/homebrew/Caskroom/miniconda/base/envs/datasci/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Log densities of first 10 training samples: [-1592.76194419 -1592.76194419 -1592.76194419 -1592.76194419
 -1592.76194419 -1592.76194419 -1592.76194419 -1592.76194419
 -1592.76194419 -1592.76194419]
KNN Accuracy : 0.9705
